自然语言处理中有两种文本摘要生成方法：抽取式和抽象式，尽管抽象式摘要的表现更好，但开发相关算法需要复杂但深度学习技巧和语言模型，为了获得合理产出，抽象式摘要方法必须能够解决诸多自然语言处理问题，如自然语言生成，语义表征和推理排序。

* 第一步：将这段话转换成句子
* 第二步：文本处理，移除停止词，数字，标点符号以及句子中的其他特殊字符。句子成分的过滤有助于移除冗余和不重要的信息。
* 第三步：分词
* 第四步：评估单词的加权出现频率（用每个单词的出现频率除以这段话中出现最多次的单词的频率）
* 第五步：用相应的加权频率替代原句中的各个单词，然后计算总和

In [ ]:
# 第一步：准备数据
import bs4 as BeautifulSoup
import urllib.request

# fetched_data = urllib3.request.urlopen('https://en.wikipedia.org/wiki/20th_century')
fetched_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/20th_century')
article_read = fetched_data.read()
article_parse = BeautifulSoup.BeautifulSoup(article_read,'html.parse')

paragraphs = article_parse.find_all('p')
article_content = ''
for p in paragraphs:
    article_content += p.text